## CS 210: Data Management for Data Science
### Lecture 12

## <font color="brown">Pandas,CSVs</font>

In [1]:
import pandas as pd
import csv

Suppose we have an auto dataset with the following fields:
* mpg
* cylinders
* displacement
* horsepower
* weight
* acceleration
* model year
* origin
* car name

Let's get the mpg, cylinders, year and name

In [2]:
for i,line in enumerate(open("auto-mpg-original.txt")):
    flds = line.split()
    out_flds = []
    out_flds.append(flds[0])
    out_flds.append(flds[1][:-1])
    out_flds.append('19' + flds[6][:-1])
    out_flds.append(flds[8])
    if i<8:
        print(','.join(out_flds))

18.0,8,1970,"chevrolet
15.0,8,1970,"buick
18.0,8,1970,"plymouth
16.0,8,1970,"amc
17.0,8,1970,"ford
15.0,8,1970,"ford
14.0,8,1970,"chevrolet
14.0,8,1970,"plymouth


But wait, the fields are separated by spaces, and the car name may contain spaces. The character used to separate the fields is called the *delimiter*. It's important to choose a character that does not appear in the data, otherwise it will be more difficult to read. 

Let's reformat this file to use a comma (",") as the delimiter. 
The file suffix will be .csv (for Comma-Separated Values). 

In [3]:
names=["mpg","cylinders","displacement",
       "horsepower","weight","acceleration",
       "model year", "origin", "car name"]

In [4]:
with open("auto-mpg-reformatted.csv", "w") as autoout:
    autoout.write(",".join(names)+"\n")
    for line in open("auto-mpg-original.txt"):
        flds = line.split()
        out_flds = []
        out_flds.append(flds[0])
        out_flds.append(flds[1][:-1])
        out_flds.append(','.join(flds[2:6]))
        out_flds.append('19' + flds[6][:-1])
        out_flds.append(flds[7][:-1])
        out_flds.append(' '.join(flds[8:]))
        autoout.write(','.join(out_flds)+'\n')

Let's find any rows with NA values. 

* `csv.DictReader`
* `csv.DictWriter`

In [5]:
reader = csv.DictReader(open('auto-mpg-reformatted.csv'))
with open('auto_mpg.csv','w') as csvfile:
    csvfile.write(','.join(reader.fieldnames)+'\n')
    for row in reader:
        if 'NA' in row.values():
            continue
        values = list(row.values())
        csvfile.write(','.join(values)+'\n')

In [6]:
with open('auto-mpg-reformatted.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    with open('auto_mpg.tsv','w',newline='') as csvout:
        writer = csv.DictWriter(csvout,
            fieldnames=reader.fieldnames,
            delimiter='\t')
        writer.writeheader()
        for row in reader:
            if 'NA' in row.values():
                continue
            writer.writerow(row)

That was a fairly easy fix. Every missing value was indicated with "NA", and every line had the same number of fields. Unfortunately, it's not always that easy. 

In [7]:
with open('iris-messy.csv') as irisfile:
    reader = csv.reader(irisfile)
    next(reader) # skip over header
    for num,row in enumerate(reader):
        if len(row) != 5:
            print(f'Row {(num+1):03} is {row}')


Row 009 is ['4.4', '2', '9', '1.4', '0.2', 'Iris-setosa']
Row 064 is ['6.1', '4.7', '1.4', 'Iris-versicolor']
Row 078 is ['6.7', '3.0', '4.5', '1.7', '6.5', 'Iris-versicolor']
Row 103 is ['7', '1', '3.0', '5.9', '2.1', 'Iris-virginica']
Row 113 is ['6.8', '3.0', '5.5', '2.1']
Row 152 is []


In [8]:
with open('iris-messy.csv') as irisfile:
    reader = csv.reader(irisfile)
    next(reader)
    for num,row in enumerate(reader):
        if len(row) < 5:
            print(f"Too few fields: {row}")
        elif len(row) > 5:
            print(f"Too many fields: {row}")
        else:
            for val in row[:-1]:
                try:
                    float(val)
                except:
                    print(f"Non-numeric value: {row}")


Too many fields: ['4.4', '2', '9', '1.4', '0.2', 'Iris-setosa']
Non-numeric value: ['4.8', 'N/A', '1.4', '0.1', 'Iris-setosa']
Non-numeric value: ['4.9', '3.1', 'n/a', '0.1', 'Iris-setosa']
Non-numeric value: ['5.0', 'na', '1.2', '0.2', 'Iris-setosa']
Non-numeric value: ['?', '3.2', '1.3', '0.2', 'Iris-setosa']
Too few fields: ['6.1', '4.7', '1.4', 'Iris-versicolor']
Non-numeric value: ['5.6', '2.5', '3.9', 'NA', 'Iris-versicolor']
Non-numeric value: ['6.8', '2.8', '?', '1.4', 'Iris-versicolor']
Too many fields: ['6.7', '3.0', '4.5', '1.7', '6.5', 'Iris-versicolor']
Too many fields: ['7', '1', '3.0', '5.9', '2.1', 'Iris-virginica']
Too few fields: ['6.8', '3.0', '5.5', '2.1']
Non-numeric value: ['6.2', '2.8', '4x8', '1.8', 'Iris-virginica']
Non-numeric value: ['6.3', '3.4', '5.6', '?', 'Iris-virginica']
Non-numeric value: ['', '3.0', '5.2', '2.0', 'Iris-virginica']
Too few fields: []


In [9]:
with open('iris-cleaned.csv','w') as outfile:
    with open('iris-messy.csv') as irisfile:
        reader = csv.reader(irisfile)
        row = next(reader)
        outfile.write(','.join(row))
        outfile.write('\n')
        for num,row in enumerate(reader):
            if len(row) != 5:
                continue
            outrow = []
            for val in row[:-1]:
                try:
                    float(val)
                    outrow.append(val)
                except:
                    outrow.append('NA')
            outrow.append(row[-1])
            outfile.write(','.join(outrow))
            outfile.write('\n')


### Reading CSV files with Pandas

* `pandas.read_csv`
* `DataFrame.to_csv`

(also: excel, html, json, pickle, sql, xml, and more)

In [10]:
df = pd.read_csv("auto-mpg-reformatted.csv", header=0, index_col=None, na_values=["NA"])

In [11]:
df.dropna()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,1970,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,1970,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,1970,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,1970,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,1970,1,ford torino
...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790.0,15.6,1982,1,ford mustang gl
402,44.0,4,97.0,52.0,2130.0,24.6,1982,2,vw pickup
403,32.0,4,135.0,84.0,2295.0,11.6,1982,1,dodge rampage
404,28.0,4,120.0,79.0,2625.0,18.6,1982,1,ford ranger


In [12]:
df.dropna().to_csv("auto-mpg-filtered.csv", index=None)

In [13]:
auto_df = pd.read_csv("auto-mpg-filtered.csv")

In [14]:
auto_df.shape

(392, 9)

#### JSON

JSON stands for JavaScript Object Notation
* serializable format for objects
* based on JavaScript’s object literals
* uses key-value pairs
https://www.json.org/json-en.html

```
{
    "people": [
        {
            "name": "Alice",
            "age": 25,
            "city": "Miami",
            "state": "FL"
        },
        {
            "name": "Bob",
            "age": 30,
            "city": "Seattle",
            "state": "WA"
        }
    ]
}
```

In [15]:
import json

with open('data.json') as f:
    data = json.load(f)

for p in data['people']:
    print("{}: {}".format(p['name'], p['age']))


Alice: 25
Bob: 30


#### JSON Data Types

* `Number`
* `String`
* `Boolean`
* `Array`
* `Object`
* `null`

Loading JSON from a string

In [16]:
json1 = '{"Hill Center":"Busch", "AB":"College Ave"}'
dict1 = json.loads(json1)
print(dict1)

{'Hill Center': 'Busch', 'AB': 'College Ave'}


Not as flexible as Python -- keys must be double-quoted strings. 

In [17]:
json2 = "{'Hill Center':'Busch', 'AB':'College Ave'}"
dict2 = json.loads(json2)
print(dict2)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

Python dictionaries can be converted to JSON. 

In [18]:
d = { 'name' : 'Jane', 'age' : 25, 'city' : 'Chicago'}
s = json.dumps(d)
print(s)


{"name": "Jane", "age": 25, "city": "Chicago"}


Will d2 be the same as d? 

In [19]:
d2 = json.loads(s)

In [20]:
d2

{'name': 'Jane', 'age': 25, 'city': 'Chicago'}

Arrays in JSON. 

In [21]:
s = '{"name": "Alice", "quiz_scores":[98,100,86,80,92]}'
d = json.loads(s)
print(d['quiz_scores'][2])

86


Typically, you will find JSON files with arrays of dictionaries. 
```
[
    {"name": "Alice", "age": 25, "city": "Chicago"},
    {"name": "Bob", "age": 30, "city": "Seattle"}
]
```

In [24]:
json_str = '[{"name": "Alice", "age": 25, "city": "Chicago"},{"name": "Bob", "age": 30, "city": "Seattle"}]'
array_of_dict = json.loads(json_str)

Use `indent` parameter to "pretty print" JSON output. 

In [25]:
print(json.dumps(array_of_dict, indent=4))

[
    {
        "name": "Alice",
        "age": 25,
        "city": "Chicago"
    },
    {
        "name": "Bob",
        "age": 30,
        "city": "Seattle"
    }
]


JSON formatted this way is readily converted to a pandas `DataFrame`:

In [26]:
pd.read_json(json_str)

,name,age,city
0,Alice,25,Chicago
1,Bob,30,Seattle


#### Exercise 1

Write a function that will take a json string like the above as a parameter, returning
a pandas Series with the name as the index and the age as a value. 

#### Generators

https://wiki.python.org/moin/Generators

Python provides generator functions as a convenient shortcut to building iterators. 

Note: a generator will provide performance benefits only if we do not intend to use that set of generated values more than once.

In [33]:
def get_multiples_list(k=3, n=10000000):
    return [i for i in range(n) if i%k==0]

In [34]:
def get_multiples_generator(k=3, n=10000000):
    i = k
    while i<n:
        if i%k==0:
            yield i
        i += 1

The first function builds a large list in memory, then returns it. This can be a bottleneck for large datasets or on virtual hardware with less memory. 

The second function returns elements one-at-a-time. This is often useful for large datasets, so that processing can begin while data is being retrieved. It can also be useful if only a small sample is required. 

In [35]:
def sample_multiples(sample_size=100):
    g = get_multiples_generator()
    return [next(g) for i in range(sample_size)]

The `next` function works with iterators and generators. 

In [36]:
def sample_multiples_list(sample_size=100):
    lst = get_multiples_list()
    return lst[:sample_size]